# Covariates
This code extracts all covariate data from FHS/ARIC follow up files and places into mastersheet. 

In [1]:
# Load packages 

import numpy as np
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import matplotlib.pyplot as plt

In [2]:
# Load mastersheet for PIDs with exposure/mediator/outcome data
filepath = "mastersheet.xlsx"
spo2_df = pd.read_excel(filepath, sheet_name="SPO2Final")
eeg_df = pd.read_excel(filepath, sheet_name="EEGFinal")
dementia_fhs_df = pd.read_excel(filepath, sheet_name="DementiaFHS")
dementia_aric_df = pd.read_excel(filepath, sheet_name="DementiaARIC")

### Tier 1 Covariates

In [3]:
# Pull in the basic covariates spreadsheet, shhs-harmonized-dataset-0.21.0
basic_covariates = pd.read_csv("datasets_FHS_ARIC_CHS_SHHS\\shhs-harmonized-dataset-0.21.0.csv")

In [4]:
# Standardize PID columns 
dementia_fhs_df["PID"] = dementia_fhs_df["PID"].astype(str).str.strip()
dementia_aric_df["PID"] = dementia_aric_df["PID"].astype(str).str.strip()
spo2_df["PID"] = spo2_df["PID"].astype(str).str.strip()
eeg_df["PID"] = eeg_df["PID"].astype(str).str.strip()

pids_spo2 = set(spo2_df['PID'])
pids_eeg = set(eeg_df['PID'])
pids_dementia = set(dementia_fhs_df['PID']).union(set(dementia_aric_df['PID']))

# Create pids_final for participants we have all exposure/mediator/outcome data for
pids_final = pids_spo2 & pids_eeg & pids_dementia
print(f"# participants with X, M, outcome data: {len(pids_final)}")

# participants with X, M, outcome data: 2439


In [5]:
# Map nsrrid onto PID, since the covariates are stored with nsrrid 
# spo2_df already has the PID:nsrrid map 
id_map = spo2_df[['PID', 'nsrrid']]

basic_covariates = basic_covariates.merge(id_map, on='nsrrid', how='left')

# Reorder so PID comes immediately after nsrrid
cols = basic_covariates.columns.tolist()
cols.insert(cols.index('nsrrid') + 1, cols.pop(cols.index('PID')))
basic_covariates = basic_covariates[cols]

# Print the result
print(basic_covariates.columns)

Index(['nsrrid', 'PID', 'visitnumber', 'nsrr_age', 'nsrr_age_gt89', 'nsrr_sex',
       'nsrr_race', 'nsrr_ethnicity', 'nsrr_bmi', 'nsrr_bp_systolic',
       'nsrr_bp_diastolic', 'nsrr_current_smoker', 'nsrr_ever_smoker',
       'nsrr_ahi_hp3u', 'nsrr_ahi_hp3r_aasm15', 'nsrr_ahi_hp4u_aasm15',
       'nsrr_ahi_hp4r', 'nsrr_ttldursp_f1', 'nsrr_phrnumar_f1',
       'nsrr_flag_spsw', 'nsrr_ttleffsp_f1', 'nsrr_ttllatsp_f1',
       'nsrr_ttlprdsp_s1sr', 'nsrr_ttldursp_s1sr', 'nsrr_ttldurws_f1',
       'nsrr_pctdursp_s1', 'nsrr_pctdursp_s2', 'nsrr_pctdursp_s3',
       'nsrr_pctdursp_sr', 'nsrr_ttlprdbd_f1'],
      dtype='object')


In [6]:
# Copy only SHHS-1 data 
basic_covariates_v1 = basic_covariates[basic_covariates['visitnumber'] == 1].copy()

# Copy only relevant columns 
tier1_cov = basic_covariates_v1[basic_covariates_v1['PID'].isin(pids_final)][['PID', 'nsrr_age', 'nsrr_sex', 'nsrr_race', 'nsrr_bmi']].copy()

print(tier1_cov.columns)

Index(['PID', 'nsrr_age', 'nsrr_sex', 'nsrr_race', 'nsrr_bmi'], dtype='object')


### Tier 2 Covariates

In [7]:
# Pull other covariates from the general SHHS dataset 
all_covariates = pd.read_csv("datasets_FHS_ARIC_CHS_SHHS\\shhs1-dataset-0.21.0.csv", low_memory=False)

In [8]:
# Merge pid with pid:nsrrid map 
id_map = spo2_df[['PID', 'nsrrid']]
all_covariates = all_covariates.merge(id_map, on='nsrrid', how='left')

# Reorder so PID comes immediately after nsrrid
cols = all_covariates.columns.tolist()

# Move PID right after nsrrid
cols.insert(cols.index('nsrrid') + 1, cols.pop(cols.index('PID')))
all_covariates = all_covariates[cols]

print(all_covariates.columns)

Index(['nsrrid', 'PID', 'pptid', 'ecgdate', 'lvh3_1', 'lvh3_3', 'st4_1_3',
       'st5_1_3', 'lvhst', 'mob1',
       ...
       'eoglqual', 'chinqual', 'oximqual', 'posqual', 'lightoff', 'oximet51',
       'monitor_id', 'headbox_id', 'rcrdtime', 'psg_month'],
      dtype='object', length=1272)


In [9]:
# Standardize datatype for PID column
all_covariates["PID"] = all_covariates["PID"].astype(str).str.strip()

In [10]:
# Filter out only the variables we want (Tier 2 covariates) and subset only the final pids  
col = ['PID', 'educat', 'alcoh', 'evsmok15', 'smknow15', 'cigday15', 'avesmk15']
tier2_covariates = all_covariates[all_covariates['PID'].isin(pids_final)][col].copy()

# Apply pd.to_numeric to relevant columns
numeric_cols = col[1:]  # Skip 'PID'
tier2_covariates[numeric_cols] = tier2_covariates[numeric_cols].apply(pd.to_numeric, errors='coerce')

print(tier2_covariates.columns)
print(f"Number PIDs: {len(tier2_covariates)}")

Index(['PID', 'educat', 'alcoh', 'evsmok15', 'smknow15', 'cigday15',
       'avesmk15'],
      dtype='object')
Number PIDs: 2439


### Tier 3 Covariates (Medications)

In [11]:
# Repeat process for medications 
filtered_pids = all_covariates[all_covariates['PID'].isin(pids_final)]

# Select columns with Group 1/Group 2 meds and exclusionary meds
manual_cols = ['tca1', 'ntca1', 'slpill15', 
               'htnmed1', 'loop1', 'hctz1', 'hctzk1', 'ccbir1', 'ccbsr1', 'alpha1', 'alphad1', 'beta1', 'betad1', 'ccb1', 'ace1', 'aced1', 'vaso1', 'vasod1', 'pvdl1', 'diuret1', 'lipid1', 'warf1', 'pdei1', 'dig1', 'anar1a1', 'anar1b1', 'anar1c1', 'anar31', 'asa1','ntg1',
               'ohga1', 'insuln1', 'ostrd1', 'sympth1', 'istrd1', 'benzod1', 
               'estrgn1', 'progst1', 'premar1', 'nsaid1', 'basq1', 'niac1', 'thry1']

# Combine with PID 
final_cols = ['PID'] + manual_cols 

meds = filtered_pids[final_cols].copy()

In [12]:
# Export med counts to CSV for reference 
med_counts = {}
for col in meds.columns:
    if col == "PID":
        continue
    numeric_col = pd.to_numeric(meds[col], errors='coerce')
    value_sum = numeric_col.dropna().sum()
    med_counts[col] = value_sum

# Convert to DataFrame and save as CSV
med_counts_df = pd.DataFrame(list(med_counts.items()), columns=["Medication", "Count"])
#med_counts_df.to_csv("medication_counts.csv", index=False)

### Physical Activity (for sensitivity analysis)

In [13]:
# Starting with FHS data 
fhs_phys = "datasets_FHS_ARIC_CHS_SHHS\\dataset_Framingham_Offspring_2023b\\Datasets\\CSV\\act1_5d_v1.csv"
fhs_phys = pd.read_csv(fhs_phys)

In [14]:
# Filter relevant pids 
fhs_phys["PID"] = fhs_phys["PID"].astype(str).str.strip()
fhs_phys_filtered = fhs_phys[fhs_phys['PID'].isin(pids_final)].copy()
print(fhs_phys_filtered.shape)

(604, 320)


In [15]:
# Extract columns of interest with physical activity data 
fhs_data = list(fhs_phys_filtered.columns[25:92])

# Note: FHS dataset separates data for each activity into two columns: _2 columns store hours/week and _4 stores weeks/year 
# Each activity type is separated by letter (ie. 6A, 6B, 6C) 
filtered_fhs_cols = [col for col in fhs_data if col.endswith('_2') or col.endswith('_4')]
with_pids =['PID']
final = with_pids+filtered_fhs_cols

# Extracting columns 
fhs_data_final = fhs_phys_filtered[final].copy()

print(fhs_data_final.columns)
print(fhs_data_final.shape)

Index(['PID', 'EH_6A_2', 'EH_6A_4', 'EH_6B_2', 'EH_6B_4', 'EH_6C_2', 'EH_6C_4',
       'EH_6D_2', 'EH_6D_4', 'EH_6E_2', 'EH_6E_4', 'EH_6F_2', 'EH_6F_4',
       'EH_6G_2', 'EH_6G_4', 'EH_6H_2', 'EH_6H_4', 'EH_6I_2', 'EH_6I_4',
       'EH_6J_2', 'EH_6J_4', 'EH_6K_2', 'EH_6K_4', 'EH_6L_2', 'EH_6L_4',
       'EH_6M_2', 'EH_6M_4', 'EH_6N_2', 'EH_6N_4', 'EH_6O_2', 'EH_6O_4',
       'EH_6P_2', 'EH_6P_4', 'EH_6Q_2', 'EH_6Q_4'],
      dtype='object')
(604, 35)


In [16]:
# Repeat process for ARIC
aric_phys = "datasets_FHS_ARIC_CHS_SHHS\\dataset_ARIC_2024b\\Main_Study\\v3\\CSV\\rpac04.csv"
aric_phys = pd.read_csv(aric_phys)

In [17]:
# Filter relevant pids 
aric_phys["ID_C"] = aric_phys["ID_C"].astype(str).str.strip()
aric_phys_filtered = aric_phys[aric_phys['ID_C'].isin(pids_final)].copy()
print(aric_phys_filtered.shape)

(1813, 46)


In [18]:
# Extract columns of interest with physical activity data 
aric_cols = ['ID_C','RPAC8', 'RPAC10', 'RPAC11', 'RPAC14', 'RPAC15', 'RPAC18', 'RPAC19', 'RPAC22', 'RPAC23']
aric_data = aric_phys_filtered[aric_cols].copy()

# Note: one participant was missing data for RPAC8 (binary for exercising), despite having detailed data about sports played and hours/week played 
# Manually assign cell to Y 
aric_data.loc[aric_data["ID_C"] == "C013200", "RPAC8"] = "Y"
print(aric_data.columns)

Index(['ID_C', 'RPAC8', 'RPAC10', 'RPAC11', 'RPAC14', 'RPAC15', 'RPAC18',
       'RPAC19', 'RPAC22', 'RPAC23'],
      dtype='object')


### Export covariate data to mastersheet

In [19]:
# Merge all covariates together for Tier 1 and Tier 2
total_covariates = tier1_cov.merge(tier2_covariates, on='PID', how='left')
print(total_covariates.columns)
print(len(total_covariates))

Index(['PID', 'nsrr_age', 'nsrr_sex', 'nsrr_race', 'nsrr_bmi', 'educat',
       'alcoh', 'evsmok15', 'smknow15', 'cigday15', 'avesmk15'],
      dtype='object')
2439


In [20]:
# Add Tier 1/Tier 2 covariates to mastersheet 

#with pd.ExcelWriter("mastersheet.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    #total_covariates.to_excel(writer, sheet_name="covariates", index=False)

In [21]:
# Add medications to mastersheet

#with pd.ExcelWriter("mastersheet.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    #meds.to_excel(writer, sheet_name="medications", index=False)

In [22]:
# Add ARIC physical activity to mastersheet 

#aric_data = aric_data.rename(columns={"ID_C": "PID"})
#with pd.ExcelWriter("mastersheet.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    #aric_data.to_excel(writer, sheet_name="Aric_phys", index=False)

In [23]:
# Add FHS physical activity to mastersheet 

#with pd.ExcelWriter("mastersheet.xlsx", mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
    #fhs_data_final.to_excel(writer, sheet_name="FHS_phys", index=False)

## Exposure & Mediator Variable Stats

In [67]:
# create a new df with only what I want (ie variables + pids) 
# then find the total stats
# then use boolean mask to split female/male

spo2_variables = spo2_df[spo2_df['PID'].isin(pids_final)][["PID", "Sex", "hb_desat", "avg_spo2_no_desat_NREM"]].copy()
eeg_variables = eeg_df[eeg_df['PID'].isin(pids_final)][["PID", "gender", "SP_DENS_all_C", "SP_CHIRP_all_C"]].copy()

In [68]:
# total dataset stats

print("Hypoxic burden average:", spo2_variables["hb_desat"].mean())
print("Hypoxic burden average:", spo2_variables["hb_desat"].std())
print("hypoxic burden stats:\n", spo2_variables["hb_desat"].describe())


print("\nBaseline spo2 average:", spo2_variables["avg_spo2_no_desat_NREM"].mean())
print("baseline spo2 std:", spo2_variables["avg_spo2_no_desat_NREM"].std())
print("baseline spo2 stats:\n", spo2_variables["avg_spo2_no_desat_NREM"].describe())

print("\nSpindle chirp avg:", eeg_variables["SP_CHIRP_all_C"].mean())
print("Spindle chirp std:", eeg_variables["SP_CHIRP_all_C"].std())
print("Spindle chirp stats:\n", eeg_variables["SP_CHIRP_all_C"].describe())

print("\nSpindle density avg:", eeg_variables["SP_DENS_all_C"].mean())
print("Spindle density std:", eeg_variables["SP_DENS_all_C"].std())
print("Spindle density stats:\n", eeg_variables["SP_DENS_all_C"].describe())

Hypoxic burden average: 62.88745188891823
Hypoxic burden average: 64.71517313673645
hypoxic burden stats:
 count    2439.000000
mean       62.887452
std        64.715173
min         1.000632
25%        26.118913
50%        45.282743
75%        75.445276
max       924.123178
Name: hb_desat, dtype: float64

Baseline spo2 average: 95.17821075319834
baseline spo2 std: 1.6094624653156908
baseline spo2 stats:
 count    2439.000000
mean       95.178211
std         1.609462
min        85.092657
25%        94.275955
50%        95.334951
75%        96.287373
max        99.726355
Name: avg_spo2_no_desat_NREM, dtype: float64

Spindle chirp avg: -0.21404688650540357
Spindle chirp std: 0.1231522649061177
Spindle chirp stats:
 count    2439.000000
mean       -0.214047
std         0.123152
min        -0.750538
25%        -0.291837
50%        -0.207398
75%        -0.129731
max         0.226247
Name: SP_CHIRP_all_C, dtype: float64

Spindle density avg: 2.6022811195618982
Spindle density std: 0.945101071

In [69]:
## HYPOXIC BURDEN WITHOUT OUTLIERS since the stdev is as big as the mean 
hb = spo2_variables['hb_desat']

Q1 = hb.quantile(0.25)
Q3 = hb.quantile(0.75)
IQR = Q3 - Q1


lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR


hb_no_outliers = hb[(hb >= lower_bound) & (hb <= upper_bound)]

print(hb_no_outliers.describe())
num_outliers = hb[(hb < lower_bound) | (hb > upper_bound)].count()
print(f"\nNumber of outliers removed: {num_outliers}")

count    2269.000000
mean       49.924272
std        32.722024
min         1.000632
25%        25.059917
50%        42.629041
75%        67.844409
max       148.636770
Name: hb_desat, dtype: float64

Number of outliers removed: 170


In [70]:
## NOWWWW Calculating the stats using the masks
# Spo2 first 
males = spo2_variables["Sex"] == "Male"
females = spo2_variables["Sex"] == "Female"

# get counts 
male_indices = spo2_variables[males].index.tolist()
female_indices = spo2_variables[females].index.tolist()

print("Female:", len(female_indices))
print("Male:", len(male_indices))

Female: 1243
Male: 1196


In [71]:
# checking counts in females 
female_avgspo2 = spo2_variables.loc[females,["avg_spo2_no_desat_NREM"]]
female_hb = spo2_variables.loc[females,["hb_desat"]]

print("Stats for females\n")
print("Stats for baseline spo2:\n", female_avgspo2["avg_spo2_no_desat_NREM"].describe())

print("\nStats for hb desat:\n", female_hb["hb_desat"].describe())

Stats for females

Stats for baseline spo2:
 count    1243.000000
mean       95.499074
std         1.580399
min        85.092657
25%        94.626744
50%        95.713050
75%        96.610374
max        99.408378
Name: avg_spo2_no_desat_NREM, dtype: float64

Stats for hb desat:
 count    1243.000000
mean       50.785141
std        51.932634
min         1.000632
25%        21.057813
50%        37.171218
75%        61.308489
max       580.027117
Name: hb_desat, dtype: float64


In [72]:
# checking counts in males 
male_avgspo2 = spo2_variables.loc[males,["avg_spo2_no_desat_NREM"]]
male_hb = spo2_variables.loc[males,["hb_desat"]]

print("Stats for males\n")
print("Stats for baseline spo2:\n", male_avgspo2["avg_spo2_no_desat_NREM"].describe())

print("\nStats for hb desat:\n", male_hb["hb_desat"].describe())

Stats for males

Stats for baseline spo2:
 count    1196.000000
mean       94.844738
std         1.571784
min        87.080830
25%        93.952364
50%        94.952300
75%        95.959622
max        99.726355
Name: avg_spo2_no_desat_NREM, dtype: float64

Stats for hb desat:
 count    1196.000000
mean       75.465355
std        73.692417
min         2.996629
25%        32.928627
50%        55.174596
75%        90.760676
max       924.123178
Name: hb_desat, dtype: float64


In [73]:
## NOWWWW Calculating the stats using the masks
# EEG now  
males = eeg_variables["gender"] == "Male"
females = eeg_variables["gender"] == "Female"

# get counts 
male_indices = eeg_variables[males].index.tolist()
female_indices = eeg_variables[females].index.tolist()

print("Female:", len(female_indices))
print("Male:", len(male_indices))

Female: 1243
Male: 1196


In [74]:
# checking counts in females 
female_chirp = eeg_variables.loc[females,["SP_CHIRP_all_C"]]
female_dens = eeg_variables.loc[females,["SP_DENS_all_C"]]

print("Stats for females\n")
print("Stats for chirp:\n", female_chirp["SP_CHIRP_all_C"].describe())

print("\nStats for density:\n", female_dens["SP_DENS_all_C"].describe())

Stats for females

Stats for chirp:
 count    1243.000000
mean       -0.238816
std         0.124850
min        -0.750538
25%        -0.316770
50%        -0.230424
75%        -0.148096
max         0.140194
Name: SP_CHIRP_all_C, dtype: float64

Stats for density:
 count    1243.000000
mean        2.767478
std         0.920442
min         0.247423
25%         2.065291
50%         2.774297
75%         3.416771
max         5.916279
Name: SP_DENS_all_C, dtype: float64


In [75]:
# checking counts in mnales
male_chirp = eeg_variables.loc[males,["SP_CHIRP_all_C"]]
male_dens = eeg_variables.loc[males,["SP_DENS_all_C"]]

print("Stats for males\n")
print("Stats for chirp:\n", male_chirp["SP_CHIRP_all_C"].describe())

print("\nStats for density:\n", male_dens["SP_DENS_all_C"].describe())

Stats for males

Stats for chirp:
 count    1196.000000
mean       -0.188305
std         0.115932
min        -0.535262
25%        -0.264765
50%        -0.184071
75%        -0.110248
max         0.226247
Name: SP_CHIRP_all_C, dtype: float64

Stats for density:
 count    1196.000000
mean        2.430592
std         0.940153
min         0.041237
25%         1.734578
50%         2.366703
75%         3.115865
max         5.002370
Name: SP_DENS_all_C, dtype: float64


## Dementia

In [76]:
## NOWWWW Moving on to DEMENTIA COUNTS 
dementia_fhs_df["PID"]=dementia_fhs_df["PID"].astype(str).str.strip()
dementia_aric_df["PID"]=dementia_aric_df["PID"].astype(str).str.strip()

dementia1 = dementia_fhs_df[dementia_fhs_df["PID"].isin(pids_final)][["PID", "fhs_dementia_from_psg", "fhs_dod_from_psg"]].copy()
dementia2 = dementia_aric_df[dementia_aric_df["PID"].isin(pids_final)][["PID", "aric_dementia_from_psg", "aric_dod_from_psg"]].copy()

print(len(dementia1)+len(dementia2))

2439


In [77]:
dementia_total = pd.concat([dementia1, dementia2], ignore_index=True)
print(dementia_total)

          PID  fhs_dementia_from_psg  fhs_dod_from_psg  \
0     1003513                    NaN               NaN   
1     1010008                    NaN               NaN   
2     1014966                    NaN               NaN   
3     1022330                    NaN            6899.0   
4     1023218                    NaN               NaN   
...       ...                    ...               ...   
2434  C015763                    NaN               NaN   
2435  C015769                    NaN               NaN   
2436  C015770                    NaN               NaN   
2437  C015775                    NaN               NaN   
2438  C015784                    NaN               NaN   

      aric_dementia_from_psg  aric_dod_from_psg  
0                        NaN                NaN  
1                        NaN                NaN  
2                        NaN                NaN  
3                        NaN                NaN  
4                        NaN                NaN  
...

In [78]:
for pid in dementia_total['PID']:
    if pid not in covariates['PID'].values:
        print(f"Missing PID in covariates: {pid}")

In [79]:
duplicate_pids = dementia_total[dementia_total['PID'].duplicated(keep=False)]
print(duplicate_pids)

Empty DataFrame
Columns: [PID, fhs_dementia_from_psg, fhs_dod_from_psg, aric_dementia_from_psg, aric_dod_from_psg]
Index: []


In [81]:
# Great now I have my dementia_total dataset but want to just create two new columns that has the value from the days to diag, and the days to death
dementia_total['days_to_diag_combined'] = dementia_total['fhs_dementia_from_psg'].combine_first(dementia_total['aric_dementia_from_psg'])
dementia_total['dod_combined'] = dementia_total['fhs_dod_from_psg'].combine_first(dementia_total['aric_dod_from_psg'])

print(dementia_total)

          PID  fhs_dementia_from_psg  fhs_dod_from_psg  \
0     1003513                    NaN               NaN   
1     1010008                    NaN               NaN   
2     1014966                    NaN               NaN   
3     1022330                    NaN            6899.0   
4     1023218                    NaN               NaN   
...       ...                    ...               ...   
2434  C015763                    NaN               NaN   
2435  C015769                    NaN               NaN   
2436  C015770                    NaN               NaN   
2437  C015775                    NaN               NaN   
2438  C015784                    NaN               NaN   

      aric_dementia_from_psg  aric_dod_from_psg  days_to_diag_combined  \
0                        NaN                NaN                    NaN   
1                        NaN                NaN                    NaN   
2                        NaN                NaN                    NaN   
3      

In [85]:
print(dementia_total["days_to_diag_combined"].notna().sum())
print(dementia_total["aric_dementia_from_psg"].notna().sum()+dementia_total["fhs_dementia_from_psg"].notna().sum())


print(dementia_total["dod_combined"].notna().sum())
print(dementia_total["aric_dod_from_psg"].notna().sum()+dementia_total["fhs_dod_from_psg"].notna().sum())


486
486
1117
1117


In [87]:
print((dementia_total["days_to_diag_combined"]).value_counts(dropna=False))

days_to_diag_combined
NaN       1953
6108.0       2
4265.0       2
5611.0       2
5888.0       2
          ... 
6047.0       1
3495.0       1
3842.0       1
7477.0       1
7123.0       1
Name: count, Length: 467, dtype: int64


In [88]:
print((dementia_total["dod_combined"]).value_counts(dropna=False))

dod_combined
NaN       1322
8311.0       3
7585.0       3
3335.0       3
8433.0       3
          ... 
8716.0       1
644.0        1
7872.0       1
6415.0       1
7682.0       1
Name: count, Length: 1051, dtype: int64


In [90]:
# copying in the sex for each PID so that way I can also split by sex for the descriptive stats
sex = covariates[["PID", "nsrr_sex"]].copy()
#dementia_total = dementia_total.merge(sex, on="PID", how="left")
#print(dementia_total)

In [91]:
# total dataset stats 
print("Total dataset days to dementia diagnosis\n")
print(dementia_total["days_to_diag_combined"].describe(), "\n")

print("Total dataset days to death\n")
print(dementia_total["dod_combined"].describe())

Total dataset days to dementia diagnosis

count     486.000000
mean     6018.059671
std      1751.341849
min     -1409.000000
25%      5138.500000
50%      6119.000000
75%      7312.500000
max      8935.000000
Name: days_to_diag_combined, dtype: float64 

Total dataset days to death

count    1117.000000
mean     5523.830797
std      2194.947018
min        68.000000
25%      3900.000000
50%      5919.000000
75%      7344.000000
max      9007.000000
Name: dod_combined, dtype: float64


In [93]:
q1 = dementia_total["dod_combined"].quantile(0.25)
median = dementia_total["dod_combined"].median()
q3 = dementia_total["dod_combined"].quantile(0.75)
print(f"Death Date: {median} days [{q1} - {q3}]")


Death Date: 5919.0 days [3900.0 - 7344.0]


In [95]:
q1 = dementia_total["days_to_diag_combined"].quantile(0.25)
median = dementia_total["days_to_diag_combined"].median()
q3 = dementia_total["days_to_diag_combined"].quantile(0.75)
print(f"PSG to dementia diagnosis: {median} days [{q1} - {q3}]")

PSG to dementia diagnosis: 6119.0 days [5138.5 - 7312.5]


###### Done: 
1. Total stats
2. Figure out nan value counts

Next   
4. separate male/female and do same thing
5. then do the censoring and figure out how many died and ded only
6. can also add a stat for how many were diagnosed after the median but we will include those for the og stats above 

In [96]:
# males v females 
males = dementia_total["nsrr_sex"] == "male"
females = dementia_total["nsrr_sex"] == "female"

# get counts 
male_indices = dementia_total[males].index.tolist()
female_indices = dementia_total[females].index.tolist()

print("Female:", len(female_indices))
print("Male:", len(male_indices))

Female: 1243
Male: 1196


In [99]:
male_dementia = dementia_total.loc[males,["days_to_diag_combined", "dod_combined"]]

print("Stats for males\n")
print("Stats for days to diagnosis:\n", male_dementia["days_to_diag_combined"].describe(),"\n")

print("Stats for days to death:\n", male_dementia["dod_combined"].describe())


Stats for males

Stats for days to diagnosis:
 count     228.000000
mean     5900.833333
std      1884.537707
min     -1409.000000
25%      4896.750000
50%      6043.000000
75%      7269.250000
max      8889.000000
Name: days_to_diag_combined, dtype: float64 

Stats for days to death:
 count     633.000000
mean     5313.824645
std      2248.419402
min        68.000000
25%      3625.000000
50%      5623.000000
75%      7132.000000
max      9007.000000
Name: dod_combined, dtype: float64


In [100]:
female_dementia = dementia_total.loc[females,["days_to_diag_combined", "dod_combined"]]

print("Stats for females\n")
print("Stats for days to diagnosis:\n", female_dementia["days_to_diag_combined"].describe(),"\n")

print("Stats for days to death:\n", female_dementia["dod_combined"].describe())


Stats for females

Stats for days to diagnosis:
 count     258.000000
mean     6121.655039
std      1621.238692
min       712.000000
25%      5358.500000
50%      6137.000000
75%      7341.500000
max      8935.000000
Name: days_to_diag_combined, dtype: float64 

Stats for days to death:
 count     484.000000
mean     5798.487603
std      2093.676686
min       138.000000
25%      4402.000000
50%      6203.000000
75%      7547.750000
max      8987.000000
Name: dod_combined, dtype: float64


In [102]:
print("Number participants with date of death:", sum(dementia_total["dod_combined"].notna()))
print("Number participants with dementia diaganosis", sum(dementia_total["days_to_diag_combined"].notna()))

Number participants with date of death: 1117
Number participants with dementia diaganosis 486


In [103]:
median_diag = dementia_total["days_to_diag_combined"].median()
condition1 = dementia_total["days_to_diag_combined"] >= median_diag
count = condition1.sum()
print("Number diagnosed on or after median date:", count)

Number diagnosed on or after median date: 243


In [105]:
condition1 = dementia_total["days_to_diag_combined"] < median_diag

print("Number participants who were diagnosed BEFORE median diagnosis date:", (condition1).sum())


Number participants who were diagnosed BEFORE median diagnosis date: 243


In [106]:
no_diagnosis_count = (dementia_total["days_to_diag_combined"].isna() & dementia_total["dod_combined"].notna()).sum()
print(f"No diagnosis but has DoD: {no_diagnosis_count}")

No diagnosis but has DoD: 743


In [108]:
condition1 = (dementia_total["dod_combined"].notna() & dementia_total["days_to_diag_combined"].notna())
condition2 = (dementia_total["dod_combined"] < dementia_total["days_to_diag_combined"])

print(f"Participants who died before diagnosis: {(condition1 & condition2).sum()}")


Participants who died before diagnosis: 0


In [116]:
diagnosis_and_dod = (dementia_total["days_to_diag_combined"].notna() & dementia_total["dod_combined"].notna()).sum()
print(f"No diagnosis but has DoD: {diagnosis_and_dod}")

No diagnosis but has DoD: 374


In [109]:
dementia_total["Time_Diag_to_Death"]=dementia_total["dod_combined"]-dementia_total["days_to_diag_combined"]

print(dementia_total["Time_Diag_to_Death"].describe())

count     374.000000
mean     1117.184492
std      1082.128421
min         1.000000
25%       209.250000
50%       743.500000
75%      1745.000000
max      6292.000000
Name: Time_Diag_to_Death, dtype: float64


In [110]:
q1 = dementia_total["Time_Diag_to_Death"].quantile(0.25)
median = dementia_total["Time_Diag_to_Death"].median()
q3 = dementia_total["Time_Diag_to_Death"].quantile(0.75)
print(f"Diag to death: {median} days [{q1} - {q3}]")

Diag to death: 743.5 days [209.25 - 1745.0]


In [113]:
#print(dementia_total["Time_Diag_to_Death"].value_counts().loc[[0, 1]])

count_first_year = dementia_total["Time_Diag_to_Death"].between(0, 365).sum()

print(f"Participants who died in first year after diagnosis: {count_first_year}")


Participants who died in first year after diagnosis: 130


In [114]:
female_timebetween = dementia_total.loc[females,["Time_Diag_to_Death"]]

print("Stats for females\n")
print("Stats for days diag to death:\n", female_timebetween["Time_Diag_to_Death"].describe(),"\n")


Stats for females

Stats for days diag to death:
 count     190.000000
mean     1207.363158
std      1119.183588
min         1.000000
25%       243.750000
50%       948.000000
75%      1748.000000
max      6026.000000
Name: Time_Diag_to_Death, dtype: float64 



In [115]:
male_timebetween = dementia_total.loc[males,["Time_Diag_to_Death"]]

print("Stats for males\n")
print("Stats for days diaag to death:\n", male_timebetween["Time_Diag_to_Death"].describe(),"\n")


Stats for males

Stats for days diaag to death:
 count     184.000000
mean     1024.065217
std      1037.301589
min        70.000000
25%       196.000000
50%       557.000000
75%      1715.750000
max      6292.000000
Name: Time_Diag_to_Death, dtype: float64 



## Missing values

In [117]:
## just to see how many people we have ALL the necessary data for. 
## that means: number for Spo2/EEG data
## a new number for basic covariates like: sex, race, age, bmi 
## then stage 2 ones for smoking/alcohol/education
## then medications 

# so we want the number we have it ALL for, then get specific numbers for what we are missing 

In [118]:
# Merge on PID to get only those with both EEG and SpO₂ data
combined = pd.merge(eeg_variables, spo2_variables, on="PID", how="inner")

# Count unique PIDs
unique_pid_count = combined["PID"].nunique()

print(f"Number of participants with both EEG and SpO₂ data: {unique_pid_count}")


Number of participants with both EEG and SpO₂ data: 2439


In [119]:
# Define EEG and SpO₂ variable column names (excluding PID)
eeg_cols = ["SP_DENS_all_C", "SP_CHIRP_all_C"]
spo2_cols = ["hb_desat", "avg_spo2_no_desat_NREM"]

# Keep only rows where all EEG and SpO₂ variables have data
valid_data = combined[eeg_cols + spo2_cols].notna().all(axis=1)

# Count unique PIDs with complete data
valid_pid_count = combined.loc[valid_data, "PID"].nunique()

print(f"PIDs with complete EEG and SpO₂ data: {valid_pid_count}")

PIDs with complete EEG and SpO₂ data: 2439


In [120]:
covariates

,PID,nsrr_age,nsrr_sex,nsrr_race,nsrr_bmi,educat,alcoh,evsmok15,smknow15,cigday15,avesmk15
0,C008484,58,male,white,25.648918,4.0,2.0,1.0,0.0,NaN,NaN
1,C008924,60,female,white,41.312358,3.0,0.0,0.0,NaN,NaN,NaN
2,C005855,59,female,white,45.681366,2.0,1.0,1.0,1.0,30.0,NaN
3,C012638,72,female,white,32.841987,2.0,0.0,1.0,0.0,NaN,NaN
4,C000847,63,female,white,23.530366,3.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2434,C002237,66,male,white,25.693134,2.0,12.0,0.0,NaN,NaN,NaN
2435,C001603,54,male,white,26.730372,3.0,20.0,1.0,1.0,8.0,NaN
2436,C003532,55,male,white,24.228571,4.0,0.0,0.0,NaN,NaN,NaN
2437,C005508,53,male,white,25.374484,2.0,0.0,0.0,NaN,NaN,NaN


In [121]:
# all covariates individually 
for col in covariates.columns:
    if col == "PID":
        continue
    else: 
        valid_data = covariates[col].notna()
        valid_pid_count = covariates.loc[valid_data, "PID"].nunique()
        print(f"# of unique PIDs with data for {col}: {valid_pid_count}")


# of unique PIDs with data for nsrr_age: 2439
# of unique PIDs with data for nsrr_sex: 2439
# of unique PIDs with data for nsrr_race: 2439
# of unique PIDs with data for nsrr_bmi: 2438
# of unique PIDs with data for educat: 2413
# of unique PIDs with data for alcoh: 2430
# of unique PIDs with data for evsmok15: 2434
# of unique PIDs with data for smknow15: 1417
# of unique PIDs with data for cigday15: 261
# of unique PIDs with data for avesmk15: 0


In [122]:
# now stage 1 
cols1 = covariates.loc[:, "nsrr_age":"nsrr_bmi"].columns

valid_data = covariates[cols1].notna().all(axis=1)

valid_pid_count = covariates.loc[valid_data, "PID"].nunique()

print(f"PIDs with all stage 1 covariates: {valid_pid_count}")

PIDs with all stage 1 covariates: 2438


In [123]:
# now stage 2 
cols2 = ["educat", "alcoh", "evsmok15"]

valid_data = covariates[cols2].notna().all(axis=1)

valid_pid_count = covariates.loc[valid_data, "PID"].nunique()

print(f"PIDs with all stage 2 covariates: {valid_pid_count}")

PIDs with all stage 2 covariates: 2399


In [124]:
# stage 1 and 2 combined: 

cols_1and2 = list(cols1)+cols2 

valid_data = covariates[cols_1and2].notna().all(axis=1)

valid_pid_count = covariates.loc[valid_data, "PID"].nunique()

print(f"PIDs with all stage 1 and 2 covariates: {valid_pid_count}")

PIDs with all stage 1 and 2 covariates: 2398


In [125]:
# okay now time to pull in medications. this will prob be messy. 
medications = pd.read_excel(filepath, sheet_name="medications")

#need to check in the two groups as well 
group2 = medications.loc[:,"htnmed1":"ntg1"].columns

valid_data = medications[group1].notna().all(axis=1)
valid_pid_count = medications.loc[valid_data, "PID"].nunique()

print(f"PIDs with group 1 medication data: {valid_pid_count}")

PIDs with group 1 medication data: 2431


In [126]:
# okay now time to pull in medications. this will prob be messy. 
medications = pd.read_excel(filepath, sheet_name="medications")

#need to check in the two groups as well 
group2 = medications.loc[:,"htnmed1":"ntg1"].columns

valid_data = medications[group2].notna().all(axis=1)
valid_pid_count = medications.loc[valid_data, "PID"].nunique()

print(f"PIDs with group 2 medication data: {valid_pid_count}")

PIDs with group 2 medication data: 2439


In [127]:
print(medications["slpill15"].value_counts(dropna=False))

slpill15
0.0    2285
1.0     134
8.0      12
NaN       8
Name: count, dtype: int64


In [128]:
medications = pd.read_excel(filepath, sheet_name="medications")


group3 = ['ohga1', 'insuln1', 'ostrd1', 'sympth1', 'istrd1', 'benzod1']


valid_data = medications[group3].notna().all(axis=1)
valid_pid_count = medications.loc[valid_data, "PID"].nunique()

print(f"PIDs with group 3 medication data: {valid_pid_count}")

PIDs with group 3 medication data: 2439


In [129]:
# now to see the numbers actually taking the meds 

taking_any = (medications[group1] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking any group1 medication: {unique_pid_count}")


Unique PIDs taking any group1 medication: 265


In [130]:
taking_any = (medications[group2] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking any group1 medication: {unique_pid_count}")


Unique PIDs taking any group1 medication: 1444


In [131]:
taking_any = (medications[group3] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking any group1 medication: {unique_pid_count}")


Unique PIDs taking any group1 medication: 336


#### Breaking down group 2 

'htnmed1', 'loop1', 'hctz1', 'hctzk1', 'ccbir1', 'ccbsr1', 'alpha1', 'alphad1', 'beta1', 'betad1', 'ccb1', 
'ace1', 'aced1', 'vaso1', 'vasod1', 'pvdl1', 'diuret1',
'lipid1', 'warf1', 'pdei1', 'dig1', 'anar1a1', 'anar1b1', 'anar1c1', 'anar31', 'asa1','ntg1',


Notes on diff groupings:
* HTN med is 823, aspiring 929, and all others is 1026

In [684]:
cardiac_renal = ['htnmed1', 'diuret1','alpha1', 'alphad1', 'beta1', 'betad1', 'ccb1','ace1', 'aced1','loop1', 'hctz1', 'hctzk1', 'ccbir1', 'ccbsr1', 'anar1a1', 'anar1b1', 'anar1c1', 'anar31', 'dig1']
blood = ['vaso1', 'vasod1', 'pvdl1', 'warf1', 'pdei1','ntg1']
asp = ['asa1']
lipids = ['lipid1']

In [685]:
taking_any = (medications[cardiac_renal] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking any htn medication: {unique_pid_count}")

Unique PIDs taking any htn medication: 871


In [686]:
taking_any = (medications[blood] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking any aspirin medication: {unique_pid_count}")

Unique PIDs taking any aspirin medication: 132


In [687]:
taking_any = (medications[asp] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking all others: {unique_pid_count}")

Unique PIDs taking all others: 929


In [689]:
taking_any = (medications[lipids] == 1.0).any(axis=1)

unique_pid_count = medications.loc[taking_any, "PID"].nunique()
print(f"Unique PIDs taking all others: {unique_pid_count}")

Unique PIDs taking all others: 342


## Okay moving back to the number of pids we have across the board

In [132]:
# first, medication group 1 and 2 
group1 = ['tca1', 'ntca1', 'slpill15']
group2 = medications.loc[:,"htnmed1":"ntg1"].columns

meds = group1 + list(group2)
valid_data = medications[meds].notna().all(axis=1)
valid_pid_count = medications.loc[valid_data, "PID"].nunique()

print(f"PIDs with group 1 and 2 medication data: {valid_pid_count}")

PIDs with group 1 and 2 medication data: 2431


In [133]:
cov_col = ["PID"] + cols_1and2
med_col = ["PID"] + meds

total_cov = covariates[cov_col]
total_meds = medications[med_col]

total = pd.merge(total_cov, total_meds, on="PID", how="left")

In [134]:
cols = total.loc[:,"nsrr_age":"ntg1"].columns

valid_data = total[cols].notna().all(axis=1)

valid_pid_count = total.loc[valid_data, "PID"].nunique()

print(f"PIDs with all stage 1-3 covariates: {valid_pid_count}")

PIDs with all stage 1-3 covariates: 2393


## and now for physical activity 
except i thinkthe code in prepping_mediatino_dataset might be more accurate for this part since i look at mismatched data too 

In [136]:
aric_phys = pd.read_excel(filepath, sheet_name="Aric_phys")
fhs_phys = pd.read_excel(filepath, sheet_name="FHS_phys")

In [139]:
# here we need to sum together the values for variables ending in _2 into a new column
activity_cols = fhs_phys.filter(regex='_2$', axis=1)
# Sum across those columns for each row
fhs_phys['total_activity_score'] = activity_cols.sum(axis=1)

relevant_fhs = fhs_phys[["PID", "total_activity_score"]].copy()

In [140]:
# Step 1: Get PIDs with any non-NaN data in aric_phys
aric_valid = aric_phys.drop(columns="PID").notna().any(axis=1)
aric_pids = set(aric_phys.loc[aric_valid, "PID"])

# Step 2: Get PIDs with any non-NaN data in relevant_fhs
fhs_valid = relevant_fhs.drop(columns="PID").notna().any(axis=1)
fhs_pids = set(relevant_fhs.loc[fhs_valid, "PID"])

# Step 3: Union of PIDs with any data in either set
combined_pids = aric_pids | fhs_pids

# Step 4: Count unique PIDs
print(f"Unique PIDs with physical activity data in ARIC or FHS: {len(combined_pids)}")


Unique PIDs with physical activity data in ARIC or FHS: 2417


In [141]:
dup_pids = aric_phys[aric_phys.duplicated(subset="PID")]
print(f"Duplicate PIDs: {len(dup_pids)}")
print(dup_pids["PID"].value_counts())

Duplicate PIDs: 0
Series([], Name: count, dtype: int64)


In [142]:
len(aric_phys)

1813

In [143]:
604+1813

2417

In [144]:
missing_rows = aric_phys[aric_phys[["RPAC8"]].isna().any(axis=1)]
print(missing_rows["PID"])


Series([], Name: PID, dtype: object)


In [145]:
# getting a set that has unique pids in aric or fhs with data that we need 

valid_aric = aric_phys[aric_phys["RPAC8"].notna() & aric_phys["PID"].notna()]
valid_fhs = relevant_fhs[relevant_fhs["total_activity_score"].notna() & relevant_fhs["PID"].notna()]

phys_pids = set(valid_aric["PID"]) | set(valid_fhs["PID"])

In [146]:
# total has ALLL the other covariates, getting set here 
complete_total = total.drop(columns="PID").notna().all(axis=1)
complete_pids = set(total.loc[complete_total, "PID"])

print(len(complete_pids))
#2399 is also correct for covariates stages 1-3

2393


In [147]:
print(len(phys_pids)) 
# 2417 pids with activity data which is right

2417


In [148]:

# 4. Intersect all three sets to get final PIDs
final_pids = complete_pids & phys_pids

# 5. Print count
print(f"Unique PIDs with data in covariates, medications, and physical activity: {len(final_pids)}")


Unique PIDs with data in covariates, medications, and physical activity: 1800


In [149]:
# Filter covariates to only final_pids
final_covariates = covariates[covariates["PID"].isin(final_pids)]

# Count values in the 'nsrr_race' column
race_counts = final_covariates["nsrr_race"].value_counts()

print(race_counts)


nsrr_race
white    1800
Name: count, dtype: int64


In [150]:
aric_phys_all = phys[["PID","RPAC8", "RPAC10"]].copy()
aric_phys_all = aric_phys_all.loc["0":"1812", :]
print(aric_phys_all)

NameError: name 'phys' is not defined

In [795]:
# Find rows where RPAC8 is 'Y' but RPAC10 is NaN
violations = aric_phys_all[(aric_phys_all["RPAC8"] == "Y") & (aric_phys_all["RPAC10"].isna())]

# Print results
print(f"Number of rows where RPAC8 == 'Y' but RPAC10 is missing: {len(violations)}")
print(violations[["PID", "RPAC8", "RPAC10"]])


Number of rows where RPAC8 == 'Y' but RPAC10 is missing: 0
Empty DataFrame
Columns: [PID, RPAC8, RPAC10]
Index: []
